In [1]:
import findspark
findspark.init('/home/anycaroliny/spark-3.3.2-bin-hadoop3')

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('log_reg').getOrCreate()

23/04/05 11:15:16 WARN Utils: Your hostname, anycaroliny-VirtualBox resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
23/04/05 11:15:16 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/05 11:15:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df = spark.read.csv('titanic.csv', inferSchema=True, header=True)
df.createOrReplaceTempView('titanic')
new_df = spark.sql("SELECT * FROM titanic")

In [4]:
new_df.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [5]:
df.columns
columns = df.select(['Survived',
 'Pclass',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Fare',
 'Embarked'])

In [6]:
final_data = columns.na.drop()

In [7]:
from pyspark.ml.feature import (VectorAssembler,
                               VectorIndexer,
                               OneHotEncoder,
                               StringIndexer)

In [8]:
gender_indexer = StringIndexer(inputCol='Sex', outputCol='SexIndex')
gender_encoder = OneHotEncoder(inputCol='SexIndex', outputCol='SexVector')

In [9]:
embark_indexer = StringIndexer(inputCol='Embarked', outputCol='EmbarkIndex')
embark_encoder = OneHotEncoder(inputCol='EmbarkIndex', outputCol='EmbarkVector')

In [10]:
assembler = VectorAssembler(inputCols=['Pclass', 'SexVector', 'EmbarkVector', 'Age', 'SibSp', 'Parch', 'Fare'],
                           outputCol='features')

In [11]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline

In [12]:
log_reg_titanic = LogisticRegression(featuresCol='features',labelCol='Survived')

In [13]:
pipeline = Pipeline(stages=[gender_indexer, embark_indexer,
                            gender_encoder, embark_encoder,
                            assembler, log_reg_titanic])

In [14]:
training_data, test_data = final_data.randomSplit([0.7, 0.3])

In [15]:
fit_model = pipeline.fit(training_data)

23/04/05 11:15:32 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
23/04/05 11:15:32 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS


In [21]:
results = fit_model.transform(test_data)
type(fit_model)

pyspark.ml.pipeline.PipelineModel

In [22]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [23]:
my_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',labelCol='Survived')

In [24]:
results.select('Survived', 'prediction').show()

+--------+----------+
|Survived|prediction|
+--------+----------+
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       0.0|
|       0|       1.0|
|       0|       1.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       1.0|
+--------+----------+
only showing top 20 rows



In [25]:
AUC = my_eval.evaluate(results)

In [26]:
AUC

0.7563322368421053